# Applied Data Science Capstone Project
## Week 3 assignment
##### Segmenting and Clustering Neighborhoods in the city of Toronto, Canada

# Part A

## Create a dataframe of neighborhood data for Toronto, Canada: postal codes, borough names, and neighborhood names

## Import needed libraries

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

print("Imports done")

Imports done


## Get the neighborhood data from Wikipedia, which has a table of the boroughs, neighborhoods, and postal codes of Toronto


In [2]:
response = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")

print("The response data has been retrieved")

The response data has been retrieved


## Use BeautifulSoup for parsing the response data.
##### Working assumption is that the table we want is the first table in the response.

In [3]:
soup = BeautifulSoup(response.content, 'lxml')
table = soup.find('table')

## Convert the HTML table to a Pandas DataFrame

In [4]:
df_tor = pd.read_html(str(table), header=0)[0]
df_tor.head(10)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


## Clean the data per the assignment specification

* The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

* Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

* If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

* More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

* Clean your Notebook and add Markdown cells to explain your work and any assumptions you are making.

* In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

## Set the column names to 'PostalCode', 'Borough', and 'Neighborhood

In [5]:
column_headers = ['PostalCode', 'Borough', 'Neighborhood']
df_tor.columns = column_headers
df_tor.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


## Only process the cells that have an assigned borough. Ignore cells with a borough that is 'Not assigned'.

In [6]:
# Remove rows where 'Borough' is 'Not assigned'. Don't forget to re-index
# the rows!
df_tor = df_tor[df_tor['Borough'] != 'Not assigned'].reset_index(drop=True)
df_tor.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Not assigned
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


## If a cell has a borough but a 'Not assigned' neighborhood, then change the neighborhood to the name of its borough.

In [7]:
df_tor['Neighborhood'].loc[df_tor['Neighborhood'] == 'Not assigned'] = df_tor['Borough']
df_tor.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


## Combine neighborhoods with the same postal code into a single row, with the neighborhood names separated by a comma and a space, e.g., 'Neighborhood1, Neighborhood2'.

In [8]:
# I sort by the values first, so the joined neighborhoods appear in alphabetic order.
df_tor = df_tor.sort_values(['PostalCode', 'Neighborhood']).groupby(['PostalCode', 'Borough'])['Neighborhood'].apply(', '.join).reset_index()

## Display the shape of the dataframe

In [9]:
df_tor.shape

(103, 3)